In [1]:
# imports 
import tensorflow as tf
import numpy as np
import os, sys
import monai

import matplotlib.pyplot as plt


2023-12-05 14:15:54.977872: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 14:15:55.085452: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 14:15:55.085538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 14:15:55.087027: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 14:15:55.096871: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 14:15:55.097786: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [44]:
def parse_tfrecord_fn(example):
    feature_description = {
            # image size, dimensions of 3 consecutive slices
            'dsize_dim0': tf.io.FixedLenFeature([], tf.int64), # 256
            'dsize_dim1': tf.io.FixedLenFeature([], tf.int64), # 256
            'dsize_dim2': tf.io.FixedLenFeature([], tf.int64), # 3
            # label size, dimension of the middle slice
            'lsize_dim0': tf.io.FixedLenFeature([], tf.int64), # 256
            'lsize_dim1': tf.io.FixedLenFeature([], tf.int64), # 256
            'lsize_dim2': tf.io.FixedLenFeature([], tf.int64), # 1
            # image slices of size [256, 256, 3]
            'data_vol': tf.io.FixedLenFeature([], tf.string),
            # label slice of size [256, 256, 1]
            'label_vol': tf.io.FixedLenFeature([], tf.string)}
    
    example = tf.io.parse_single_example(example, feature_description)
    return example



def decode_fn(example):
    # Decode string to bytes
    d0 = tf.io.cast(example['dsize_dim0'], tf.int32)
    data_vol = tf.io.decode_raw(example['data_vol'], tf.float32)
    label_vol = tf.io.decode_raw(example['label_vol'], tf.float32)

    print(data_vol.shape)
    print(label_vol.shape)

    # Reshape the data to the desired shape [256, 256, 3]
    data_vol = tf.image.resize(data_vol, (example['dsize_dim0'], example['dsize_dim1'], example['dsize_dim2']))
    
    # Reshape the label to the desired shape [256, 256, 1]
    label_vol = tf.image.resize(label_vol, (example['lsize_dim0'], example['lsize_dim1'], example['lsize_dim2']))


    print(data_vol.shape)
    print(label_vol.shape)

    return data_vol, label_vol


In [40]:
def visualize_tfrecord(tfrecord_path, num_samples=1):
    dataset = tf.data.TFRecordDataset(tfrecord_path)

# Parse the TFRecords file using the parse_tfrecord_fn
    parsed_dataset = dataset.map(parse_tfrecord_fn)

    # Decode and preprocess the data using the decode_fn
    decoded_dataset = parsed_dataset.map(decode_fn)

    # Visualize a single example
    for data_vol, label_vol in decoded_dataset.take(1):
        plt.figure(figsize=(10, 5))

        # Display the input image
        plt.subplot(1, 2, 1)
        plt.imshow(data_vol)
        plt.title('Input Image')

        # Display the label image
        plt.subplot(1, 2, 2)
        plt.imshow(tf.squeeze(label_vol), cmap='gray')
        plt.title('Label Image')

    plt.show()

In [45]:
# Specify the path to your TFRecord file
tfrecord_path = '../../PnpAda_release_data/train_val/mr_train_tfs/mr_train_slice100.tfrecords'

# Visualize the TFRecord
visualize_tfrecord(tfrecord_path, num_samples=1)

(None,)
(None,)


ValueError: in user code:

    File "/scratch-local/aeijpe/ipykernel_1701006/944729288.py", line 30, in decode_fn  *
        data_vol = tf.image.resize(data_vol, (example['dsize_dim0'], example['dsize_dim1'], example['dsize_dim2']))

    ValueError: 'images' must have either 3 or 4 dimensions.
